In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

cwd = os.getcwd() #
fullscrn()

In [2]:
def wndwng0d(x,snstvty=2):            #روش تشخیص ایپاک پایه
    smpl_rte=480
    i=0
    wndws=np.array([])
    #for i in range (1,2):   #print(np.shape(x))
    if snstvty==1 :                        #تشخیص ایپاک پایه
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=200,prominence=300, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=160,prominence=100, width=3)
            #print('class ', i, 'Row = ', j, 'have ',len(peaks),'peaks')
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :  #win=np.append(win, i)# if len(win)==smpl_rte+1 :
                        wndws=np.append(wndws, win)


    if snstvty==2 :                        #تشخیص ایپاک حساس تر
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.02*max(xj), distance=160,prominence=200, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.01*max(xj), distance=120,prominence=60, width=3)
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :
                        wndws=np.append(wndws, win)
    rows=int(len(wndws)/(smpl_rte))            #/(smpl_rte+1))
    wndws0=np.reshape(wndws,(rows,(smpl_rte)))    #(wndws,(rows,(smpl_rte+1)))
    return(wndws0)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=17
for i in range (1,18):
    vars()['ecg'+str(i)]=MITBIH(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,18):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=1000

1 283	2 66	3 20	4 135	5 13	6 21	7 133	8 55	9 13	10 10	11 10	12 10	13 11	14 103	15 62	16 10	17 45	

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])
for cls in range (1,18):                                 #ساخت پنجره های داده های آموزش اصلی
    dta=np.array(vars()['ecg'+str(cls)][int(.1*len(vars()['ecg'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   

print("each class and its windows =")
cls_wndws=np.array([])
for cls in range (1,18):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 2598	2 788	3 224	4 1564	5 158	6 189	7 1371	8 553	9 114	10 104	11 72	12 107	13 156	14 1014	15 633	16 45	17 450	
 max = 2598


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
for X in range(1,6):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                              #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2

            f_scr=np.array([57,65,74,75,74,62,60,74])     #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            #f_scr=np.array([51,68,73,74,71,56,55,70])      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            scors0=np.array([28,4,71,67,0,75,16,86,56,0,40,81,0,99,99,99,99])         #Class Recalls without Augmentation
            #scors0=np.array([[28,4,71,67,0,75,16,86,56,0,40,81,0,99,100,100,100])     #Class Recalls without Augmentation
            MxMn20fscr=((max(f_scr)-f_scr[0])/20)**0.7
            AgScr=1+alpha*MxMn20fscr*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific
            
            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            #print(aug_amnt)
            rprt=np.empty((0,3),float)
            print(aug_amnt)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,Aug_data(cwd,i,'00',cls_wndws[i-1]),axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        xtrain=np.append(xtrain,Aug_data(cwd,i,10,aug_amnt[i-1,0]),axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-3
                        dp_slct_dstrb=2
                        xtrain=np.append(xtrain,Aug_data(cwd,i,20,aug_amnt[i-1,1]),axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-3
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,30,aug_amnt[i-1,2]),axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-1
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,40,aug_amnt[i-1,3]),axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-3
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,50,aug_amnt[i-1,4]),axis=0)
                    if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-3
                        dp_slct_dstrb=4                             
                        xtrain=np.append(xtrain,Aug_data(cwd,i,60,aug_amnt[i-1,5]),axis=0) #Scale
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-3
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,70,aug_amnt[i-1,6]),axis=0)
                             
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])

                #for i in range (1,18):             #نرمالسازی داده های افزایشی
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

                wndws_test=np.empty((0,smpl_rte+1), float)
                smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
                i=0
                windws=np.array([])
                for cls in range (1,18):                                 #ساخت پنجره های داده های آموزش اصلی
                    dta=np.array(vars()['ecg'+str(cls)][:int(.1*len(vars()['ecg'+str(cls)])),:]) #انتخاب محدوده ترِین جهت داده افزایی
                    vars()['wndws_tst'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
                    clm0vlu=np.zeros(len(vars()['wndws_tst'+str(cls)]))
                    cls_clm=np.array(clm0vlu)
                    cls_clm=np.int16(cls_clm+cls)
                    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
                    vars()['wndws_tst'+str(cls)]=np.concatenate((vars()['wndws_tst'+str(cls)],cls_clm),axis=1)   

                #print(np.max(wndws_tst))
                for i in range (1,18):                # Normalization test windows
                    cls=i
                    '''print('\n cls', i, ' >> ')
                    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))'''
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    '''print('after normalizing >>')
                    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))'''

                #print("each class and its windows =")                    #تجمیع کلاس های تست
                cls_wndws=np.array([])
                for cls in range (1,18):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    cls_wndws=np.append(cls_wndws,wns)
                    #print(cls, wns, end='\t')
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)

                #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
                #print('\n max instance in classes of test windows =', mx_wndws)

                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                #VGG16_1D

                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :

                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data



                model_crs = Sequential()
                #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
                model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
                model_crs.add(BatchNormalization())
                model_crs.add(MaxPooling1D(pool_size=2,strides=3))

                model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
                model_crs.add(BatchNormalization())
                model_crs.add(MaxPooling1D(pool_size=2,strides=2))

                model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
                model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
                model_crs.add(MaxPooling1D(pool_size=2,strides=2))

                model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
                model_crs.add(MaxPooling1D(pool_size=2,strides=2))

                model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
                #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


                model_crs.add(Flatten())
                model_crs.add(Dropout(0.1))
                model_crs.add(Dense(units=512,activation="relu"))
                #model_crs.add(Dense(units=2048,activation="relu"))
                model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

                #_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

                #print('np.shape(X_test)=',np.shape(X_test))

                #print('Accuracy= ', accuracy)

                #y_pred_vgg = model_crs.predict_classes(X_test)
                predict_x=model_crs.predict(X_test)              # Function 1

                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

                #y_pred_crs = model_crs.predict_classes(X_test)

                cm = confusion_matrix(y_test, y_pred_crs)
                #print(cm)
                acc3=accuracy_score(y_test, y_pred_crs)

                #print(classification_report(y_test, y_pred_crs))

                #sns.heatmap(cm, annot=True)

                #plt.imshow(cm)

                ### Maxcount Predicting Test data
                ecg_test=np.array(ecg1[:int(.1*len(ecg1)),:])
                #print(1,len(ecg_test), end='\t')
                for i in range (2,18):
                    ecg_test=np.concatenate((ecg_test,vars()['ecg'+str(i)][:int(.1*len(vars()['ecg'+str(i)])),:]),axis=0)
                    #print(i,len(vars()['ecg'+str(i)]), end='\t')    
                #print('\n ecg_test shape = ',np.shape(ecg_test))
                i=0
                mlpinputshps=np.array([])
                maxcntlbl=np.array([])
                mlpinput=np.array([])
                mlplbl=np.array([])
                cnnlbl=np.array([])
                peaklens=np.array([])            #array of peaks quantity in each raw signal
                x=np.array([])
                wow=0

                for i in range (len(ecg_test)): #calculating max peaks in row signals
                    peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
                maxpeaks=int(np.max(peaklens))
                #print(maxpeaks)

                prd_lbls_tst=np.empty((0,maxpeaks), int)              #have peaks(windows) lables for each raw signal

                for i in range (len(ecg_test)):
                    windws=np.array([])
                    yyy=np.array([])
                    yy=np.array([])
                    win=np.array([])
                    x=np.array(ecg_test[i,:-1])
                    windws=wndwng0d(x)
                    windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
                    windws = np.expand_dims(windws, axis=2)
                    yy= model_crs.predict(windws)#yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
                    yyy=np.int16(maxindx(yy))         # one-hot to class lable
                    maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
                    zers=np.zeros(int(maxpeaks-len(yyy)))
                    x=np.array(np.append(yyy, zers))
                    prd_lbls_tst=np.append(prd_lbls_tst, [x], axis=0)
                maxcntlbl=np.int16(maxcntlbl)                           
                maxcntlbl=np.int16(maxcntlbl)

                lbls=np.int16(ecg_test[:,-1])

                '''cm = confusion_matrix(lbls, maxcntlbl)
                print(cm)
                acc3=accuracy_score(lbls, maxcntlbl)
                print(classification_report(lbls, maxcntlbl))

                predict_x=model_crs.predict(X_test)# Function 1
                y_pred_crs=maxindx(predict_x)      # function from augment.py to remove 0 index predictions'''

                rprt0=classification_report(lbls, maxcntlbl,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 2599
[[ 164  348  368  348  102  348   61]
 [ 388  825  874  825  243  825  146]
 [ 289  615  651  615  181  615  108]
 [ 172  366  388  366  108  366   65]
 [ 457  971 1028  971  285  971  171]
 [ 284  602  638  602  177  602  106]
 [ 306  651  689  651  191  651  115]
 [ 224  477  505  477  140  477   84]
 [ 334  709  751  709  209  709  125]
 [ 462  981 1039  981  289  981  173]
 [ 374  795  842  795  234  795  140]
 [ 278  590  625  590  174  590  104]
 [ 457  971 1028  971  286  971  171]
 [ 151  322  341  322   95  322   57]
 [ 187  398  421  398  117  398   70]
 [ 243  515  546  515  152  515   91]
 [ 204  434  460  434  128  434   77]]
1/1 [==============================] - 0s 12ms/step
rslts= X 1  alpha 1  Beta 1  average= [97 99 97]  max= [97 99 97]  var= [0 0 0]
[[ 176  793  890  793   69  793   25]
 [ 236 1066 1195 1066   92 1066   33]
 [ 137  620  695  620   54  620   19]
 [ 134  607  681  607   53  607   19]
 [ 248 1120 1255 1120   97 1120   35]
 [ 135  612  686  

1/1 [==============================] - 0s 13ms/step
rslts= X 1  alpha 1  Beta 2  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]
[[   6  594  836  594    0  594    0]
 [  11 1000 1409 1000    1 1000    0]
 [   6  580  818  580    0  580    0]
 [   6  554  780  554    0  554    0]
 [  12 1130 1592 1130    1 1130    0]
 [   6  583  822  583    0  583    0]
 [   8  743 1047  743    0  743    0]
 [   6  575  810  575    0  575    0]
 [   6  590  831  590    0  590    0]
 [  12 1132 1595 1132    1 1132    0]
 [   7  612  862  612    0  612    0]
 [   6  584  823  584    0  584    0]
 [  12 1132 1595 1132    1 1132    0]
 [   6  557  785  557    0  557    0]
 [   6  573  807  573    0  573    0]
 [   6  587  827  587    0  587    0]
 [   6  578  814  578    0  578    0]]
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step
rslts= X 1  alpha 1  Beta 6  average= [97 98 97]  max= [98 99 98]  var= [0 0 0]
[[ 542 1153 1220 1153  339 1153  203]
 [ 674 1433 1517 1433  421 1433  253]
 [ 373  793  839  793  233  793  140]
 [ 380  808  855  808  238  808  143]
 [ 697 1481 1568 1481  436 1481  261]
 [ 356  757  801  757  223  757  134]
 [ 611 1298 1375 1298  382 1298  229]
 [ 303  643  681  643  189  643  114]
 [ 444  943  998  943  277  943  166]
 [ 698 1483 1570 1483  436 1483  262]
 [ 517 1098 1162 1098  323 1098  194]
 [ 329  700  741  700  206  700  123]
 [ 698 1483 1571 1483  436 1483  262]
 [ 236  502  532  502  148  502   89]
 [ 243  516  546  516  152  516   91]
 [ 249  528  559  528  155  528   93]
 [ 245  520  551  520  153  520   92]]
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step
rslts= X 1  alpha 2  Beta 1  average= [98 99 98]  max= [100 100 100]  var= [0 0 0]
[[ 238 1074 1204 1074   93 1074   33]
 [ 347 1565 1754 1565  135 1565   49]
 [ 147  666  747  666   58  666   21]
 [ 147  666  747  666   58  666   21]
 [ 368 1661 1862 1661  144 1661   52]
 [ 143  646  724  646   56  646   20]
 [ 290 1310 1469 1310  113 1310   41]
 [ 131  591  662  591   51  591   18]
 [ 175  790  886  790   68  790   25]
 [ 368 1663 1864 1663  144 1663   52]
 [ 215  971 1089  971   84  971   30]
 [ 137  618  693  618   53  618   19]
 [ 368 1663 1864 1663  144 1663   52]
 [ 122  553  619  553   48  553   17]
 [ 126  568  636  568   49  568   18]
 [ 129  581  652  581   50  581   18]
 [ 127  573  642  573   50  573   18]]
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 13ms/step
rslts= X 1  alpha 2  Beta 2  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]
[[   7  670  944  670    0  670    0]
 [  15 1427 2011 1427    1 1427    0]
 [   6  581  818  581    0  581    0]
 [   6  554  781  554    0  554    0]
 [  18 1675 2361 1675    1 1675    0]
 [   6  583  822  583    0  583    0]
 [  10  935 1318  935    1  935    0]
 [   6  575  810  575    0  575    0]
 [   6  594  836  594    0  594    0]
 [  18 1677 2364 1677    1 1677    0]
 [   7  638  898  638    0  638    0]
 [   6  584  823  584    0  584    0]
 [  18 1678 2364 1678    1 1678    0]
 [   6  557  785  557    0  557    0]
 [   6  573  807  573    0  573    0]
 [   6  587  827  587    0  587    0]
 [   6  578  814  578    0  578    0]]
1/1 [==============================] - 0s 12ms/step


...

1/1 [==============================] - 0s 16ms/step
rslts= X 2  alpha 2  Beta 2  average= [98 98 98]  max= [98 99 98]  var= [0 1 0]
[[  1 106 150 106   0 106   0]
 [  4 329 464 329   0 329   0]
 [  1 133 187 133   0 133   0]
 [  1 106 149 106   0 106   0]
 [  4 397 559 397   0 397   0]
 [  1 136 191 136   0 136   0]
 [  2 196 276 196   0 196   0]
 [  1 127 179 127   0 127   0]
 [  2 140 197 140   0 140   0]
 [  4 399 562 399   0 399   0]
 [  2 151 213 151   0 151   0]
 [  1 137 193 137   0 137   0]
 [  4 399 562 399   0 399   0]
 [  1 110 155 110   0 110   0]
 [  1 125 176 125   0 125   0]
 [  2 139 196 139   0 139   0]
 [  1 130 184 130   0 130   0]]
1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step
rslts= X 2  alpha 2  Beta 6  average= [93 95 93]  max= [95 98 95]  var= [5 6 4]
[[146 311 329 311  91 311  55]
 [208 442 468 442 130 442  78]
 [102 217 230 217  64 217  38]
 [ 98 207 220 207  61 207  37]
 [219 466 493 466 137 466  82]
 [ 97 206 219 206  61 206  36]
 [180 382 405 382 112 382  67]
 [ 76 161 170 161  47 161  28]
 [129 274 290 274  81 274  48]
 [220 468 495 468 138 468  83]
 [155 330 350 330  97 330  58]
 [ 88 186 197 186  55 186  33]
 [220 468 495 468 138 468  83]
 [ 47 101 107 101  30 101  18]
 [ 54 114 121 114  34 114  20]
 [ 60 126 134 126  37 126  22]
 [ 56 119 126 119  35 119  21]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 12ms/step


KeyboardInterrupt: 

In [6]:
rslts

array(['X 1  alpha 1  Beta 1  average= [97 99 97]  max= [97 99 97]  var= [0 0 0]',
       'X 1  alpha 1  Beta 2  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 1  alpha 1  Beta 6  average= [97 98 97]  max= [98 99 98]  var= [0 0 0]',
       'X 1  alpha 2  Beta 1  average= [98 99 98]  max= [100 100 100]  var= [0 0 0]',
       'X 1  alpha 2  Beta 2  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]',
       'X 1  alpha 2  Beta 6  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]',
       'X 1  alpha 3  Beta 1  average= [100 100 100]  max= [100 100 100]  var= [0 0 0]',
       'X 1  alpha 3  Beta 2  average= [98 99 98]  max= [100 100 100]  var= [0 0 0]',
       'X 1  alpha 3  Beta 6  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 2  alpha 1  Beta 1  average= [96 96 96]  max= [97 99 97]  var= [2 5 1]',
       'X 2  alpha 1  Beta 2  average= [94 97 94]  max= [95 98 95]  var= [3 0 2]',
       'X 2  alpha 1  Beta 6  average= [94 97 94]  max= [98 99 98]  v

In [ ]:
cls_wndws[i-1]

In [ ]:
np.shape(f_scr_rtio)

In [ ]:
AgScr[i-1]

In [ ]:
os._exit(00)
srt2